## World Map with Sites ##

This demonstrates the plotting of sites and their connectivity status on a map of the world, using [Plotly Express](https://plotly.com/python/plotly-express/) to create the map, and [Gradio](https://www.gradio.app/) to create the user interface. If these are not part of your existing Python environment you will need to install them using pip, together with all other prerequisite modules.

### Preparation
First let's import the modules we need:

In [ ]:
import csv
import json
import os
import random

import gradio as gr
import pandas as pd
import plotly.express as px


#
# Helper module 
# https://github.com/catonetworks/data-analytics/blob/main/notebooks/cato.py
#
from cato import API

### Making the accountSnapshot API call ###
We're going to call accountSnapshot with just the fields we're interested in - the site names, connectivity status, configured country and tunnel interface public IP geolocation data:

In [ ]:
def accountSnapshot(C, ID):
    variables = {
        "accountID":ID
    }
    query = """query accountSnapshot($accountID:ID!) {
    	accountSnapshot(accountID:$accountID) {
    		sites {
    		  connectivityStatus
    		  info {
    			name
    			countryCode
    		  }
    		  devices {
    			interfaces {
    				tunnelRemoteIPInfo {
    					latitude
    					longitude
    				}
    			}
    		  }
    		}
    	}
    }"""
    success,snapshot = C.send("accountSnapshot", variables, query)
    if not success:
        print(f'ERROR calling accountSnapshot:{snapshot}')
    return snapshot

### Loading the default country geolocation data ###
We want to plot all sites on the map regardless of their connection status. Plotting disconnected sites in a way which is visually different from connected sites is probably useful to most CMA admins, but if a site is not connected then it doesn't have a public IP so we dont have geolocation data. How can we solve this?

There are several strategies we could employ, such as checking for past connection history and using a prior IP, but this still leaves a problem for sites which have **never** connected, such as sites awaiting deployment. Something which all configured sites have regardless of connection status or history is a configured **country**, which we can use to provide an "estimated" geolocation. If we know the co-ordinates for a geographical centre of each country, we can place disconnected sites on or near this.

We load latitude and longitude from a csv file:

In [ ]:
COUNTRIES = []
with open("cclatlong.csv","r") as file:
    for row in csv.DictReader(file):
        COUNTRIES.append(row)

### Loading the snapshot data into a Pandas dataframe ###
To make it easy for Plotly Express to add site data to the map, we need a function which will take as input the accountSnapshot result, and return as output a Pandas dataframe. The function needs to use the connectivity status field to determine for each site whether we will use the connected tunnel IP co-ordinates, or the default "centre of mass" country co-ordinates:

In [ ]:
def load_snapshot_into_dataframe(snapshot_data, country_data):
    lats = []
    longs = []
    names = []
    sizes = []
    colours = []
    for site in snapshot_data["data"]["accountSnapshot"]["sites"]:
        if site["connectivityStatus"].lower() == "connected":
            names.append(site["info"]["name"])
            lats.append(site["devices"][0]["interfaces"][0]["tunnelRemoteIPInfo"]["latitude"])
            longs.append(site["devices"][0]["interfaces"][0]["tunnelRemoteIPInfo"]["longitude"])
            colours.append("connected")
        else:
            names.append(site["info"]["name"])
            lat = 0
            longi = 0
            for row in country_data:
                if row["Alpha-2 code"] == site["info"]["countryCode"]:
                    lat = float(row["Latitude (average)"])
                    longi = float(row["Longitude (average)"])
                    break
            #
            # Add some random noise to the co-ordinates to prevent multiple disconnected sites
            # in the same country from ending up on exactly the same spot. The amount of noise
            # is independent of the size of the country, which can result in disconnected dots
            # being outside a smaller country's borders.
            #
            lats.append(lat + (random.randint(-10,+10)/5))
            longs.append(longi + (random.randint(-10,+10)/5))
            colours.append("disconnected")
        sizes.append(10)
    data = {
		"Latitude": lats,
		"Longitude": longs,
		"Site": names,
		"Size": sizes,
		"Colour": colours,
    }
    return data

### Creating the map figure ###

We're getting close to the end now. All we need to do before we add the UI is to write a function which will turn the DataFrame into a map and return it. This function will be the heart of the Gradio app.

In [ ]:
def create_map(df): 
    fig = px.scatter_mapbox(
        pd.DataFrame(df),
        lat="Latitude",
        lon="Longitude",
        text="Site",
        zoom=2,
        #height=700,
        color_discrete_map={"connected":'green',"disconnected":'red'},
        color="Colour",
        size="Size",
        size_max=10,
        hover_data={"Site":True, "Latitude":False, "Longitude":False, "Size":False, "Colour":False},
    )
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ))
    return fig

### Putting it all together as a Gradio app ###
Finally we need a function to orchestrate the data pipeline, incorporate that as the click action for a Gradio button, and launch the app.

In [ ]:
def go(ID, key):
    #
    # Create the API connection
    #
    C = API(key)    
    #
    # Get the snapshot data
    #
    snapshot = accountSnapshot(C,ID)
    #
    # Load it into a dataframe
    #
    df = load_snapshot_into_dataframe(snapshot, COUNTRIES)
    #
    # Create and return the map
    #
    return create_map(df)


#
# Gradio app UI
#
with gr.Blocks() as demo:
    with gr.Column():
        input_id = gr.Textbox(label="Cato Account ID")
        input_key = gr.Textbox(label="Cato API Key", type="password")
        plot = gr.Plot()
        button = gr.Button("Load Sites")
        button.click(go, inputs=[input_id, input_key], outputs=[plot])
demo.launch()        